<a href="https://colab.research.google.com/github/manuelapop/neural-network-simple-example/blob/main/neural-network-simple-example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
# Create a Model Class that inherits nn.Module, 2 neuron layers
class Model(nn.Module):
  # Input layer (4 features of the flower) -->	sepal.length, sepal.width, petal.length, petal.width
  # Hidden Layer1 (number of neurons) --> 8 neurons
  # H2 (n) --> 9 neurons
  # output (3 classes of iris flowers) Setosa, Versicolor, Virginica
  def __init__(self, in_features=4, h1=8, h2=9, out_features=3):
    super().__init__() # instantiate our nn.Module
    self.fc1 = nn.Linear(in_features, h1) #fc - fully connected, first layer you add in features and push it to hidden layer 1
    self.fc2 = nn.Linear(h1, h2) #hidden layer one push to hidden layer 2
    self.out = nn.Linear(h2, out_features) #out hidden layer 2 to output

  def forward(self, x): #forwarding function
    x = F.relu(self.fc1(x)) #relu pushing fc1
    x = F.relu(self.fc2(x)) #relu pushing fc2
    x = self.out(x) #relu pushing output

    return x
